# <font color ='green' > Common


In [1]:
from collections import Counter , defaultdict

import pandas as pd

pd.set_option('display.max_columns', None)
from pandas import Series as s , DataFrame as df
import numpy as np
import seaborn as sns

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt, rcParams as rc
%matplotlib inline
rc["figure.figsize"] = 10,6

from warnings import filterwarnings
filterwarnings('ignore')


from scipy import stats as s
from random import randint
from re import search
from pathlib import Path

from functools import reduce
from io import StringIO
from bs4 import BeautifulSoup

# import pprint
from pprint import pprint 
import re, pprint, random, requests, glob

from scipy.stats import mstats
from operator import itemgetter
from timeit import timeit


# <font color ='orange' > Classification


In [2]:
#preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, normalize


from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

#Modle Selection
from sklearn.model_selection import train_test_split, cross_val_score

#Evaluation Metrics
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, recall_score, precision_score, explained_variance_score
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_recall_fscore_support


In [3]:
# Load Data Set

pd.options.display.max_columns = 250
pd.options.display.max_rows = 2000
pd.options.display.float_format = '{:.2f}'.format

pd.options.display.max_columns = None
pd.options.display.max_rows = None

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)


# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

# <font color ='blue' > HighLevel


# <font color ='yellow' > NLP


In [1]:
#Importing libraries
import nltk, re, pprint, random

from nltk.tokenize import word_tokenize
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


from keras.models import Sequential,load_model
from keras.preprocessing.text import Tokenizer#break a string into tokens. 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense,LSTM,Embedding

from sklearn.model_selection import train_test_split


from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import word2vec



from nltk import FreqDist

pd.set_option('max_colwidth', 100)

import spacy
nlp = spacy.load('en')

##vvi

import nltk
nltk.download('stopwords')







# NLP

In [2]:

sentences = [
    'i love my dog',
    'I, love my cat',
    'You love my dog!',
    "I am running with dog"
]

tokenizer = Tokenizer(num_words = 1)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'i': 1, 'love': 2, 'my': 3, 'dog': 4, 'cat': 5, 'you': 6, 'am': 7, 'running': 8, 'with': 9}


# <font color ='red' > NLP problem 1 : Text


In [ ]:
count_vectorizer = CountVectorizer() # or TfidfVectorizer()

## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])


## we use .todense() here because these vectors are "sparse" (only non-zero elements are kept to save space)
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())


train_vectors = count_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])


## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
import linear_model
clf = linear_model.RidgeClassifier()

scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores


clf.fit(train_vectors, train_df["target"])

# <font color ='red' > NLP problem 2 : Text


In [ ]:

doc = nlp("Tea is healthy and calming, don't you think?")

for token in doc:
    print(token)
    
print(f"Token \t\tLemma \t\tStopword".format('Token', 'Lemma', 'Stopword'))
print("-"*40)
for token in doc:
    print(f"{str(token)}\t\t{token.lemma_}\t\t{token.is_stop}")
    

from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')



# <font color ='green' > V.V.I : Model Building, Train, Evaluate for NLP(text or spam, sentiment)


In [ ]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(train_data["Tweet"].values)

vocab_size = len(tokenizer.word_index) + 1
X = tokenizer.texts_to_sequences(train_data["Tweet"].values)
X = pad_sequences(X, maxlen=max_len, padding='post')

#pre process test data with param of train data
X_test = tokenizer.texts_to_sequences(test_data["Tweet"].values)
X_test = pad_sequences(X_test, maxlen=max_len, padding='post')

# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 300))
for word, index in tqdm(tokenizer.word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector


vector_features = 300


model = Sequential()
model.add(Embedding(vocab_size, vector_features, input_length=X.shape[1], weights=[embedding_matrix], trainable=False))
model.add(SpatialDropout1D(0.2))

model.add(Bidirectional(LSTM(300, activation='relu', dropout=0.3, recurrent_dropout=0.3), input_shape=(vector_features, vocab_size)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3, activation='softmax'))

model.summary()

epochs = 100
batch_size = 512

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_data=(X_valid, y_valid),
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

#Model Accuracy
accuracy = history.history["accuracy"]
loss = history.history["loss"]

val_accuracy = history.history["val_accuracy"]
val_loss = history.history["val_loss"]

print("training acuuracy {0}% and training loss {1}%".format(accuracy[-1]*100, loss[-1]*100))
print("validation acuuracy {0}% and validation loss {1}%".format(val_accuracy[-1]*100, val_loss[-1]*100))

#plot 

plt.plot(accuracy,'g',label='training accuracy')
plt.plot(val_accuracy, 'r', label='validation accuracy')
plt.legend()
plt.show()


plt.plot(loss,'g',label='training loss')
plt.plot(val_loss, 'r', label='validation loss')
plt.legend()
plt.show()

y_pred = model.predict(X_test)


confusion_matrix(np.argmax(y_test, 1), np.argmax(y_pred, 1))


## <font color ='blue' > create a BOW : bag of words model on the spam dataset (col : label(o/p), message(i/p)).

In [ ]:
# load data
spam = pd.read_csv("SMSSpamCollection.txt", sep = "\t", names=["label", "message"])


# convert messages into list and all rows details converting in single list
messages = [message for message in messages]
print(messages)


def preprocess(document):
    'changes document to lower case and removes stopwords'

    # change sentence to lower case
    document = document.lower()

    # tokenize into words
    print("document : ", document)
    words = word_tokenize(document)
    print("token : ", words)

    # remove stop words
    words = [word for word in words if word not in stopwords.words("english")]

    # join words to make sentence
    document = " ".join(words)
    
    return document

messages_preprocess = [preprocess(message) for message in messages]
print(messages_preprocess)


#### Creating bag of words model using count vectorizer function
vectorizer = CountVectorizer() # or TfidfVectorizer()

bow_model = vectorizer.fit_transform(documents)
print(bow_model)  # returns the rown and column number of cells which have 1 as value


print(bow_model.toarray()) # print the full sparse matrix


print(bow_model.shape)
print(vectorizer.get_feature_names())


# look at the dataframe
pd.DataFrame(bow_model.toarray(), columns = vectorizer.get_feature_names())

### <font color ='red' > BOW : bag of words model on the spam dataset (col : label(o/p),msg(i/p)) & use Stemming & lemmatising

In [ ]:

stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

# add stemming and lemmatisation in the preprocess function
def preprocess(document, stem=True):
    'changes document to lower case and removes stopwords'

    # change sentence to lower case
    document = document.lower()

    # tokenize into words
    words = word_tokenize(document)

    # remove stop words
    words = [word for word in words if word not in stopwords.words("english")]
    
    if stem:
        words = [stemmer.stem(word) for word in words]
    else:
        words = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words]
        #words = [wordnet_lemmatizer.lemmatize(word, pos='n') for word in words] # or

    # join words to make sentence
    document = " ".join(words)
    
    return document


messages = [preprocess(message, stem=True) for message in spam.message] # stem messages

messages = [preprocess(message, stem=False) for message in spam.message] # lemmatise messages




plot_word_frequency(messages, 10):


# bag of words model
vectorizer = CountVectorizer() # or TfidfVectorizer()
bow_model = vectorizer.fit_transform(messages)

# look at the dataframe
pd.DataFrame(bow_model.toarray(), columns = vectorizer.get_feature_names())

#output

#Stemm :  359 tokens after lemmatizing the messages as compared to 381 tokens without stemming.


#Lemm : 363 tokens after lemmatizing the messages as compared to 381 tokens without lemmatising. 
# But, on the other hand, stemmer reduces the token count to 359. Lemmatization doesn't work as expected 
# because the data is very unclean.


* Lemmatising is faster than stemming in this case because the nltk lemmatiser also takes another argument called the part-of-speech (POS) tag of the input word.
* The default part-of-speech tag is 'noun'..
* Right now, the stemmer will have more accuracy than the lemmatiser because each word is lemmatised assuming it's a noun. To lemmatise efficiently, you need to pass it's POS tag manually.

In [4]:
%%time
print("hi")

hi
CPU times: user 207 µs, sys: 44 µs, total: 251 µs
Wall time: 156 µs


# <font color ='red' > Model Building(Train, Evaluate)

In [ ]:
X_train = message[:70] # Train : 70% of dataset
X_test = message[70:] #  Test  : 30% of dataset

y_train = label[:70] # Train : 70% of dataset
y_test = label[70:] #  Test  : 30% of dataset

vect = CountVectorizer(stop_words='english',max_features=5000)
vect.fit(X_train)


X_train_transformed = vect.transform(X_train) # train : fit & transform or fit_transform
X_test_transformed  = vect.transform(X_test)  # test  : only transform


bnb = BernoulliNB() # or MultinomialNB()


bnb.fit(X_train_transformed,y_train)

# predict class
pred_train_ys = bnb.predict(X_train_transformed)
pred_test_ys = bnb.predict(X_test)

# accuracy
print("Train accuracy: ", accuracy_score(y_train, pred_train_ys))

print("Test accuracy: ", accuracy_score(y_test, pred_test_ys))

MultinomialNB work very well & high performance in accuracy better than BernoulliNB

### <font color ='red' > Word embeddings approach : 2 options(A.Use pre-trained word vectors (Glove), B.Train our own vectors) ----->      (Note : pre-trained word vectors (Glove) is better than Train our own vectors)

# <font color ='green'   > Word embeddings approach :  A) Use pre-trained word vectors 

In [ ]:
glove_input_file = 'glove.6B.200d.txt'
word2vec_output_file = 'glove.6B.200d.w2vformat.txt'

glove2word2vec(glove_input_file, word2vec_output_file)

glove_model = KeyedVectors.load_word2vec_format("glove.6B.200d.w2vformat.txt", binary=False)



## Sentence vector by averaging word vectors

In [ ]:
stop_words = stopwords.words("english")

def sent_vec(sent):
    wv_res = np.zeros(glove_model.vector_size)
    ctr = 1
    for w in sent:
        if w in glove_model:
            ctr += 1
            wv_res += glove_model[w]
    wv_res = wv_res/ctr
    #return (wv_res, ctr)
    return wv_res


train_doc_vecs = []
for doc in X:    
    doc_words = [term for term in doc if term not in stop_words]
    train_doc_vecs.append(sent_vec(doc_words))
    

test_doc_vecs = []
for doc in X_test:    
    doc_words = [term for term in doc if term not in stop_words]
    test_doc_vecs.append(sent_vec(doc_words))
    
### Building a predictive model on the averaged word vectors

logreg = LogisticRegression(penalty="l1", random_state=42, C = 3.5)
logreg.fit(train_doc_vecs,y)

pred_train_ys = logreg.predict(train_doc_vecs)
pred_test_ys = logreg.predict(test_doc_vecs)
print("Train accuracy: ", accuracy_score(pred_train_ys, y))
print("Test accuracy: ", accuracy_score(pred_test_ys, y_test))



# <font color ='green'   > Word embeddings approach :  B) Train our own vectors)

In [ ]:
### Training our own wordvectors on the data
# We'll create a combined text file to train our word vectors - more data is better. 
#Although in this case we would still have just 7.7K instances to learn from

X_comb = np.concatenate([X,X_test])
len(X_comb)
print(X_comb[6000])


w2v = word2vec.Word2Vec(X_comb, window=2, min_count=2, sg = 1, size=200)
w2v.most_similar("future", topn=5)


### Sentence vectors by averaging vectors for words

In [ ]:
def sent_vec_w2v(sent):
    wv_res = np.zeros(w2v.vector_size)
    ctr = 1
    for w in sent:
        if w in w2v:
            ctr += 1
            wv_res += w2v[w]
    wv_res = wv_res/ctr
    return wv_res

#### Getting the sentence vectors for the test and train sets

train_doc_vecs = []
for doc in X:    
    doc_words = [term for term in doc if term not in stop_words]
    train_doc_vecs.append(sent_vec_w2v(doc_words))
    
test_doc_vecs = []
for doc in X_test:    
    doc_words = [term for term in doc if term not in stop_words]
    test_doc_vecs.append(sent_vec_w2v(doc_words))
    
logreg = LogisticRegression(penalty="l1", random_state=42, C = 8)
logreg.fit(train_doc_vecs,y)

pred_train_ys = logreg.predict(train_doc_vecs)
pred_test_ys = logreg.predict(test_doc_vecs)
print("Train accuracy: ", accuracy_score(pred_train_ys, y))
print("Test accuracy: ", accuracy_score(pred_test_ys, y_test))


# <font color ='green' > plot_word_frequency graph

In [ ]:
def plot_word_frequency(words, top_n=10):
    word_freq = FreqDist(words)
    labels = [element[0] for element in word_freq.most_common(top_n)]
    counts = [element[1] for element in word_freq.most_common(top_n)]
    plot = sns.barplot(labels, counts)
    return plot



plot_word_frequency(messages or document, 10): # i/p : message or doc or text , plot after preprocessing
